In [1]:
import os, pathlib
import numpy as np
import neuropythy as ny
import matplotlib as mpl, matplotlib.pyplot as plt
import ipyvolume as ipv
import pandas as pd

In [19]:
# The following code is for loading in the ventral ROI labels.
ventral_label_path = '~/code/hcp-annot-vc_data/save'
def load_ventral_pathaddr(rater, sid, h, lbl):
    p = pathlib.Path(ventral_label_path).expanduser()
    p = p / rater / str(sid)
    return ny.load(str(p / f"{h}.{lbl}_path.json.gz"))
def load_ventral_pathaddrs(rater, sid, h):
    rois = {}
    for k in ['hV4', 'VO1', 'VO2']:
        rois[k] = load_ventral_pathaddr(rater, sid, h, k)
    return rois
def load_ventral_paths(rater, sid, h):
    addrs = load_ventral_pathaddrs(rater, sid, h)
    hem = ny.data['hcp_lines'].subjects[sid].hemis[h]
    return {k: ny.geometry.Path(hem, path)
            for (k,path) in addrs.items()}
def load_ventral_labels(rater, sid, h):
    paths = load_ventral_paths(rater, sid, h)
    lbls = {}
    for (k,path) in paths.items():
        lbl = (path.label > 0.5)
        if np.sum(lbl) > np.sum(~lbl):
            lbl = ~lbl
        lbls[k] = lbl
    return lbls

In [20]:
# This is the sample rater and subject/hemisphere we will be using.
rater = 'lindazelinzhao'
sid = 102311
h = 'lh'

# Get the subject and hemisphere object, and the midgray surface.
sub = ny.data['hcp_lines'].subjects[sid]
hem = sub.hemis[h]
surf = hem.surface('midgray')

# Load the labels for visual areas and combine them.
ventral_labels = load_ventral_labels(rater, sid, h)
early_labels = {f'V{k}': hem.prop('visual_area') == k
                for k in [1,2,3]}
labels = dict(ventral_labels)
labels.update(early_labels)